In [1]:
import pandas as pd
import requests

In [2]:
def format_date(date):
    """
    Функция принимает на вход дату в формате день/месяц/год и возвращает в формате год-месяц-день
    
    Параметры:
        date: string — дата
    """
    
    day, month, year = date.split("/")
    
    return "{}-{}-{}".format(year, month, day)

In [3]:
def get_exchange_rates(date_start, date_end, save=False, filename=None):
    """
    Функция принимает на вход начало и конец периода, за которые нужно получить данные о курсе валют.
    Возвращает DataFrame со столбцами [Date, CharCode, Value]. Если указана опция save, то сохраняет
    созданный DataFrame с указанным именем в формате .xlsx
    
    Параметры:
        date_start: string — начало периода,
        date_end: string — конец периода,
        save: boolean — сохранять данные в excel-файл или нет, по умолчанию save=False,
        filename: string — если save=True, то является обязательным, название excel-файла
    """
    
    if save and not filename:
        raise ValueError("Filename should be specified when save option is True")
        
    # генерируем список дат, переводим в нужный нам формат
    date_range = pd.date_range(start=format_date(date_start), end=format_date(date_end)).strftime('%d/%m/%Y')
    exchange_rates = []
    
    for date in date_range:
        # осуществляем запрос к сайту ЦБ с указанной датой
        request_string = "http://www.cbr.ru/scripts/XML_daily.asp?date_req=" + date
        request = requests.get(request_string)
        
        # читаем полученный xml-файл, "//ValCurs/Valute" — указание на данные, которые будут строками
        df = pd.read_xml(request.text, xpath="//ValCurs/Valute", encoding="windows-1251")
        
        # приводим курс к нужному нам виду (цена за одну единицу)
        df["Value"] = df["Value"].str.replace(',', '.').astype(float)
        df["Value"] = df["Value"]/df["Nominal"]
        
        # добавляем столбец даты и убираем все ненужные
        df.insert(0, 'Date', date)
        df = df.drop(columns=["ID", "NumCode", "Nominal", "Name"])
        
        exchange_rates.append(df)
    
    # объединяем данные за все дни в один DataFrame
    exchange_rates_df = pd.concat(exchange_rates).reset_index(drop=True)
    
    if save:
        exchange_rates_df.to_excel(filename, index=False)
        
    return exchange_rates_df

In [4]:
date_start = "10/10/2022"
date_end = "18/10/2022"

In [5]:
get_exchange_rates(date_start, date_end, save=True, filename="exchange_rates.xlsx")

,Date,CharCode,Value
0,10/10/2022,AUD,39.229000
1,10/10/2022,AZN,36.027900
2,10/10/2022,GBP,68.591100
3,10/10/2022,AMD,0.151393
4,10/10/2022,BYN,24.428600
...,...,...,...
301,18/10/2022,SEK,5.466170
302,18/10/2022,CHF,61.597100
303,18/10/2022,ZAR,3.394880
304,18/10/2022,KRW,0.043032
